# 🏛️ Archimedes Chess AI - One-Click Setup

**High-Performance Chess AI with ResNet, AMP, and Advanced MCTS**

## 🚀 Quick Start

1. **Aktiviere GPU**: `Laufzeit` → `Laufzeittyp ändern` → `Hardwarebeschleuniger: GPU` (T4 empfohlen)
2. **Führe die Setup-Zelle aus** (unten)
3. **Fertig!** Das Training startet automatisch

## ✨ Features (v2.1)

- **ResNet-basiertes TPN**: 10 Residual Blocks mit Batch Normalization
- **Automatic Mixed Precision (AMP)**: 2-3x schnelleres Training auf unterstützter Hardware
- **Time-based Iterative Deepening**: Dynamische Suchtiefensteuerung
- **Strategic Abstraction Network (SAN)**: GNN-basierte langfristige Planung
- **LRU Transposition Table**: Effiziente Cache-Verwaltung
- **WandB Integration**: Live-Metriken und Visualisierung

---

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════
# 🎯 ONE-CLICK SETUP & TRAINING
# ═══════════════════════════════════════════════════════════════════════════
# Diese Zelle macht ALLES:
# 1. Installiert Dependencies
# 2. Klont das Repository
# 3. Überprüft GPU-Verfügbarkeit
# 4. Startet das Training mit optimalen Parametern
# ═══════════════════════════════════════════════════════════════════════════

import os
import sys
import subprocess
from pathlib import Path

print("\n" + "="*70)
print("🏛️  ARCHIMEDES CHESS AI - ONE-CLICK SETUP")
print("="*70 + "\n")

# ─────────────────────────────────────────────────────────────────────────
# STEP 1: Repository Setup
# ─────────────────────────────────────────────────────────────────────────
print("📂 [1/4] Setting up repository...\n")

REPO_URL = "https://github.com/Bademeischta/Projekt-Archimedes.git"
REPO_NAME = "archimedes"

if not Path("train_end_to_end.py").exists():
    if not Path(REPO_NAME).exists():
        print(f"   Cloning from: {REPO_URL}")
        !git clone {REPO_URL} {REPO_NAME}
    
    %cd {REPO_NAME}
    print("✅ Repository setup complete!\n")
else:
    print("✅ Project files already present!\n")

# ─────────────────────────────────────────────────────────────────────────
# STEP 2: Install Dependencies
# ─────────────────────────────────────────────────────────────────────────
print("📦 [2/4] Installing dependencies...\n")

try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

if IN_COLAB:
    # Standard deps from requirements file
    if Path("requirements_colab.txt").exists():
        !pip install -q -r requirements_colab.txt
    else:
        !pip install -q chess python-chess torch-geometric tqdm psutil wandb pynvml
    
    print("✅ Dependencies installed!\n")
else:
    print("⚠️  Not running in Colab. Assuming dependencies are already installed.\n")

# ─────────────────────────────────────────────────────────────────────────
# STEP 3: System Check
# ─────────────────────────────────────────────────────────────────────────
print("🔍 [3/4] Checking system...\n")

import torch

print("   System Information:")
print(f"   ├─ Python: {sys.version.split()[0]}")
print(f"   ├─ PyTorch: {torch.__version__}")
print(f"   └─ CUDA Available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"\n   🎮 GPU Information:")
    print(f"   ├─ Name: {torch.cuda.get_device_name(0)}")
    print(f"   ├─ CUDA Version: {torch.version.cuda}")
    props = torch.cuda.get_device_properties(0)
    print(f"   ├─ Memory: {props.total_memory / 1024**3:.1f} GB")
    print(f"   ├─ Compute Capability: {props.major}.{props.minor}")
    print(f"   └─ AMP Support: {'✅ Yes' if props.major >= 7 else '❌ No (requires CC 7.0+)'}")
    
    # Determine optimal settings based on GPU
    gpu_mem_gb = props.total_memory / 1024**3
    if gpu_mem_gb >= 15:  # T4, V100, A100
        batch_size = 16
        num_workers = 1
    else:  # Smaller GPUs
        batch_size = 8
        num_workers = 1
    
    use_amp = props.major >= 7
    
    print(f"\n   📊 Recommended Settings:")
    print(f"   ├─ Batch Size: {batch_size}")
    print(f"   ├─ Workers: {num_workers}")
    print(f"   └─ AMP: {'Enabled' if use_amp else 'Disabled'}")
else:
    print("\n   ⚠️  No GPU detected!")
    print("   Please enable GPU: Laufzeit → Laufzeittyp ändern → GPU")
    print("   Training will be VERY slow on CPU!\n")
    batch_size = 4
    num_workers = 0
    use_amp = False

print("\n✅ System check complete!\n")

# ─────────────────────────────────────────────────────────────────────────
# STEP 4: Start Training
# ─────────────────────────────────────────────────────────────────────────
print("🚀 [4/4] Starting training...\n")
print("="*70)

# Training parameters
TOTAL_GAMES = 100          # Total games to generate
TRAINING_ITERATIONS = 500  # Total training steps
WARMUP_GAMES = 20          # Initial buffer fill
SCHEDULER = "cosine"

cmd = f"""python train_end_to_end.py \
    --num-workers {num_workers} \
    --total-games {TOTAL_GAMES} \
    --training-iterations {TRAINING_ITERATIONS} \
    --batch-size {batch_size} \
    --warmup-games {WARMUP_GAMES} \
    --scheduler {SCHEDULER} \
    --checkpoint-dir ./checkpoints \
    --save-every 5 \
    {'--use-amp' if use_amp else '--no-amp'}
"""

print(f"\n📝 Training Configuration:")
print(f"   ├─ Total Games: {TOTAL_GAMES}")
print(f"   ├─ Training Iterations: {TRAINING_ITERATIONS}")
print(f"   ├─ Batch Size: {batch_size}")
print(f"   ├─ Workers: {num_workers}")
print(f"   ├─ Warmup Games: {WARMUP_GAMES}")
print(f"   ├─ Scheduler: {SCHEDULER}")
print(f"   └─ AMP: {'Enabled' if use_amp else 'Disabled'}")
print("\n" + "="*70 + "\n")

# Execute
!{cmd}

print("\n" + "="*70)
print("✅ TRAINING COMPLETE!")
print("="*70)
print("\n📁 Checkpoints: ./checkpoints/")
print("📊 Logs: ./checkpoints/training_logs.db")
print("\n💡 Run the next cells to visualize progress or test the model!\n")

---

## 📊 Optional: View Training Progress

Monitor losses and accuracies from the internal SQLite database.

In [ ]:
import sqlite3
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt

db_path = Path("./checkpoints/training_logs.db")

if db_path.exists():
    conn = sqlite3.connect(str(db_path))
    df = pd.read_sql_query("SELECT epoch, total_loss, policy_loss, value_loss, top1_accuracy FROM epoch_summary", conn)
    conn.close()
    
    if not df.empty:
        fig, ax1 = plt.subplots(figsize=(10, 5))
        
        ax1.set_xlabel('Epoch')
        ax1.set_ylabel('Loss', color='tab:red')
        ax1.plot(df['epoch'], df['total_loss'], color='tab:red', label='Total Loss')
        ax1.tick_params(axis='y', labelcolor='tab:red')
        
        ax2 = ax1.twinx()
        ax2.set_ylabel('Accuracy', color='tab:blue')
        ax2.plot(df['epoch'], df['top1_accuracy'], color='tab:blue', label='Top-1 Accuracy')
        ax2.tick_params(axis='y', labelcolor='tab:blue')
        
        plt.title('Training Progress')
        plt.show()
        
        print("\nLatest Epoch Summary:")
        print(df.tail(5).to_string(index=False))
else:
    print("⚠️  No logs found. Start training first!")

---

## 🎮 Optional: Test the Trained Model

Visualize the model's play on a virtual board.

In [ ]:
import torch
import chess
import chess.svg
from IPython.display import SVG, display, clear_output
import time
from src.archimedes.model import TPN, SAN, PlanToMoveMapper
from src.archimedes.search import ConceptualGraphSearch
from pathlib import Path

checkpoint_path = Path("./checkpoints/latest_checkpoint.pt")

if checkpoint_path.exists():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # Initialize and load
    tpn = TPN().to(device)
    san = SAN().to(device)
    mapper = PlanToMoveMapper().to(device)
    
    checkpoint = torch.load(checkpoint_path, map_location=device)
    tpn.load_state_dict(checkpoint["tpn_state_dict"])
    san.load_state_dict(checkpoint["san_state_dict"])
    mapper.load_state_dict(checkpoint["mapper_state_dict"])
    
    tpn.eval()
    san.eval()
    mapper.eval()
    
    search = ConceptualGraphSearch(tpn, san, mapper, num_simulations=100)
    
    board = chess.Board()
    
    for _ in range(10):
        if board.is_game_over(): break
        
        clear_output(wait=True)
        display(SVG(chess.svg.board(board, size=400)))
        
        result = search.search(board, temperature=0.2)
        board.push(result["best_move"])
        time.sleep(1)
        
    clear_output(wait=True)
    display(SVG(chess.svg.board(board, size=400)))
    print("Test Match Finished!")
else:
    print("⚠️  No checkpoint found!")

---

**Archimedes Chess AI** | [GitHub](https://github.com/Bademeischta/Projekt-Archimedes.git) | *Powered by ResNet & GNN Planning*